<a href="https://colab.research.google.com/github/StarWanderer1337/GridMaster/blob/main/grid_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GridMaster

> Combining gspread & pandas to use google sheet(s) like databases.

In [46]:
from gspread.worksheet import ValueRange

from google.auth import default
from google.colab import drive
from google.colab import auth

from typing import Union
from typing import List
from typing import Any

import pandas as pd

import gspread
import random

In [49]:
def read_sheet(all_values: Union[ValueRange, List[List[Any]]]) -> pd.DataFrame:
  """
  Reads data from a list of lists or a gspread ValueRange and converts it to a Pandas DataFrame.

  Args:
    all_values: A list of lists representing the sheet data, where the first list is the header row,
                or a gspread.models.ValueRange object containing the sheet data.

  Returns:
    A Pandas DataFrame containing the data from the sheet, with an added 'id' column starting from 2.
    Returns an empty DataFrame if an error occurs.
  """
  try:
      # Extract the header row
      header = all_values[0]
      # Extract the data rows
      data_rows = all_values[1:]
      # Create a Pandas DataFrame
      df = pd.DataFrame(data_rows, columns=header)
      # Add a "id" column
      df.insert(0, "id", range(2, len(all_values) + 1))

      return df

  except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")
      return pd.DataFrame(data=None)

def update_sheet(sheet: gspread.worksheet.Worksheet, df: pd.DataFrame) -> bool:
  """
  Updates a Google Sheet with the data from a Pandas DataFrame.

  Args:
    sheet: A gspread.worksheet.Worksheet object representing the sheet to be updated.
    df: A Pandas DataFrame containing the data to be written to the sheet.

  Returns:
    True if the sheet was updated successfully, False otherwise.
  """
  try:
    # Remove the previously created column id
    if "id" in df.columns:
        del df["id"]
    # Get headers and data into the same dataframe
    data = [df.columns.tolist()] + df.values.tolist()
    # Write data to cells
    sheet.update(range_name="A1", values=data)

    return True

  except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")
      return False

In [52]:
# Get credentials and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Connect to google drive
drive.mount("/content/drive", force_remount=True)

# Open a Google Sheet by title (replace "ping" with your sheet's title and "pung" with your worksheet's title)
sheet = gc.open("ping").worksheet(title="pung")

# Append a new row with a randomly generated string
sheet.append_row([f"p{random.randint(a=0, b=9)}ng"])

# Read all values from the sheet into a Pandas DataFrame
df = read_sheet(all_values=sheet.get_all_values())

# Update the value in the 'pyng' column for the row with 'id' equal to 4
# Note: Ensure a row with id 4 exists and the column 'pyng' is present in your sheet.
column_to_update = "pyng"
if 4 in df["id"].values and column_to_update in df.columns:
    df.loc[(df["id"] == 4), column_to_update] = f"p{random.randint(a=0, b=9)}ng"
else:
    print(f"Warning: Row with id 4 or column 'pyng' not found in the DataFrame. Update skipped.")

# Update the Google Sheet with the modified DataFrame
update_sheet(sheet=sheet, df=df)

# Read and display the updated data from the sheet
display(read_sheet(all_values=sheet.get_all_values()))

Mounted at /content/drive


,id,pyng
0,2,pong
1,3,peng
2,4,p8ng
3,5,p8ng
4,6,p1ng
5,7,p0ng
6,8,p0ng
7,9,p2ng
8,10,p0ng
9,11,p2ng
